In [68]:
import numpy as np
import pandas as pd

In [69]:
customer_codes = pd.read_excel('Excel_report/Azimi.xlsx', sheet_name='Codes', index_col=0)
Azimi_report = pd.read_excel('Excel_report/Azimi.xlsx', sheet_name='Report', index_col=0)
receipt = pd.read_excel('Excel_report/Report.xlsm', sheet_name='Receipt', index_col=0)
delivery = pd.read_excel('Excel_report/Report.xlsm', sheet_name='Delivery', index_col=0)

collective = pd.concat([receipt,delivery], axis=0)
collective = collective.sort_values('تاريخ میلادی')
collective = collective.reset_index()

c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: CollReport[#All].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: Table16[#All].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: CollReport[#All].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")
c:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\reader\workbook.py:118: UserWarning: Print area cannot be set to Defined name: Table16[#All].
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [70]:
collective['بدهکار'] = -(collective['مقدار (کیلوگرم)'][collective['مقدار (کیلوگرم)']<0])
collective['بستانکار'] = collective['مقدار (کیلوگرم)'][collective['مقدار (کیلوگرم)']>0]
collective = collective.fillna(0)

group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective['بدهکار تجمیعی'] = group_by_cust['بدهکار'].cumsum()
collective['بستانکار تجمیعی'] = group_by_cust['بستانکار'].cumsum()

collective['مانده در خط'] = collective['بدهکار تجمیعی'] - collective['بستانکار تجمیعی']

In [71]:
collective[collective['کد ردیابی']==10003][:20]

,تاريخ,تاريخ میلادی,کد ردیابی,مقدار (کیلوگرم),بدهکار,بستانکار,بدهکار تجمیعی,بستانکار تجمیعی,مانده در خط
70,00/01/01,2021-03-21,10003,-18547.0,18547.0,0.0,18547.0,0.0,18547.0
101,00/01/15,2021-04-04,10003,-2068.0,2068.0,0.0,20615.0,0.0,20615.0
104,00/01/16,2021-04-05,10003,1585.0,0.0,1585.0,20615.0,1585.0,19030.0
107,00/01/16,2021-04-05,10003,-3552.0,3552.0,0.0,24167.0,1585.0,22582.0
114,00/01/18,2021-04-07,10003,-1900.0,1900.0,0.0,26067.0,1585.0,24482.0
121,00/01/18,2021-04-07,10003,2837.0,0.0,2837.0,26067.0,4422.0,21645.0
129,00/01/21,2021-04-10,10003,-2010.0,2010.0,0.0,28077.0,4422.0,23655.0
141,00/01/22,2021-04-11,10003,2434.0,0.0,2434.0,28077.0,6856.0,21221.0
148,00/01/22,2021-04-11,10003,-2186.0,2186.0,0.0,30263.0,6856.0,23407.0
152,00/01/23,2021-04-12,10003,-878.0,878.0,0.0,31141.0,6856.0,24285.0


In [72]:
def not_settled_inline(collective):
    collective['مانده درخط تسویه نشده'] = collective['بدهکار تجمیعی'] - collective['بستانکار'].sum()
    collective.loc[collective['مانده درخط تسویه نشده']<0,'مانده درخط تسویه نشده'] = 0
    return collective
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective = group_by_cust.apply(not_settled_inline)

In [73]:
collective['سفارش تسویه شده'] = collective['بدهکار'] - collective['مانده درخط تسویه نشده']
collective.loc[collective['سفارش تسویه شده']<0,'سفارش تسویه شده'] = 0
collective['سفارش تسویه نشده'] = collective['بدهکار'] - collective['سفارش تسویه شده']

In [74]:
report_end_date = pd.to_datetime('2021-12-27')
credit_in_days = pd.Timedelta(days=15)

In [76]:
collective['روز از ارسال سفارش'] = pd.Timedelta(days=0)
collective.loc[collective['سفارش تسویه نشده']>0,'روز از ارسال سفارش'] = report_end_date - collective['تاريخ میلادی']

collective['تاخیر از سررسید'] = pd.Timedelta(days=0)
# collective.loc[collective['سفارش تسویه نشده']>0,'تاخیر از سررسید'] = collective.loc[collective['سفارش تسویه نشده']>0,'روز از ارسال سفارش']- credit_in_days
collective.loc[collective['سفارش تسویه نشده']>0,'تاخیر از سررسید'] = collective['روز از ارسال سفارش'] - credit_in_days

collective['روز از ارسال سفارش'] = collective['روز از ارسال سفارش'].dt.days
collective['تاخیر از سررسید'] = collective['تاخیر از سررسید'] .dt.days
collective['تسویه نشده × روز از ارسال'] = collective['سفارش تسویه نشده'] * collective['روز از ارسال سفارش']

In [46]:
# collective['سفارش سررسید شده'] = 0
collective['سفارش سررسید شده'] = collective.loc[collective['تاخیر از سررسید']>0,'سفارش تسویه نشده']
collective['سفارش سررسید شده × تاخیر از سررسید'] = collective['سفارش سررسید شده'] * collective['تاخیر از سررسید']


In [60]:
collective[collective['کد ردیابی']==10003].iloc[:,[0,1,8,11,12,13,14,15,16]][:10]

,تاريخ,تاريخ میلادی,مانده در خط,سفارش تسویه نشده,روز از ارسال سفارش,تاخیر از سررسید,تسویه نشده × روز از ارسال,سفارش سررسید شده,سفارش سررسید شده × تاخیر از سررسید
70,00/01/01,2021-03-21,18547.0,0.0,0,0,0.0,NaN,NaN
101,00/01/15,2021-04-04,20615.0,0.0,0,0,0.0,NaN,NaN
104,00/01/16,2021-04-05,19030.0,0.0,0,0,0.0,NaN,NaN
107,00/01/16,2021-04-05,22582.0,0.0,0,0,0.0,NaN,NaN
114,00/01/18,2021-04-07,24482.0,0.0,0,0,0.0,NaN,NaN
121,00/01/18,2021-04-07,21645.0,0.0,0,0,0.0,NaN,NaN
129,00/01/21,2021-04-10,23655.0,0.0,0,0,0.0,NaN,NaN
141,00/01/22,2021-04-11,21221.0,0.0,0,0,0.0,NaN,NaN
148,00/01/22,2021-04-11,23407.0,0.0,0,0,0.0,NaN,NaN
152,00/01/23,2021-04-12,24285.0,0.0,0,0,0.0,NaN,NaN


In [49]:
columns = pd.Series(collective.columns)
columns

0                                  تاريخ
1                           تاريخ میلادی
2                              کد ردیابی
3                        مقدار (کیلوگرم)
4                                 بدهکار
5                               بستانکار
6                          بدهکار تجمیعی
7                        بستانکار تجمیعی
8                            مانده در خط
9                  مانده درخط تسویه نشده
10                       سفارش تسویه شده
11                      سفارش تسویه نشده
12                    روز از ارسال سفارش
13                       تاخیر از سررسید
14             تسویه نشده × روز از ارسال
15                      سفارش سررسید شده
16    سفارش سررسید شده × تاخیر از سررسید
dtype: object

In [53]:
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
group_by_cust.sum(numeric_only=True)

,مقدار (کیلوگرم),بدهکار,بستانکار,بدهکار تجمیعی,بستانکار تجمیعی,مانده در خط,مانده درخط تسویه نشده,سفارش تسویه شده,سفارش تسویه نشده,روز از ارسال سفارش,تاخیر از سررسید,تسویه نشده × روز از ارسال,سفارش سررسید شده,سفارش سررسید شده × تاخیر از سررسید
کد ردیابی,,,,,,,,,,,,,,
10002,0.0,1502.0,1502.0,1502.0,3004.0,-1502.0,0.0,1502.0,0.0,0,0,0.0,0.0,0.0
10003,-37200.0,373634.0,336434.0,47880540.0,40101858.0,7778682.0,581721.0,336434.0,37200.0,245,5,598014.0,16877.0,130221.0
10004,-12.0,5524.0,5512.0,43361.0,41149.0,2212.0,24.0,5512.0,12.0,23,8,276.0,12.0,96.0
10007,-52.5,52.5,0.0,52.5,0.0,52.5,52.5,0.0,52.5,281,266,14752.5,52.5,13965.0
10008,-578.0,578.0,0.0,578.0,0.0,578.0,578.0,0.0,578.0,281,266,162418.0,578.0,153748.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30205,0.0,4794.0,4794.0,13121.0,19488.0,-6367.0,0.0,4794.0,0.0,0,0,0.0,0.0,0.0
50006,-7581.0,7581.0,0.0,7581.0,0.0,7581.0,7581.0,0.0,7581.0,281,266,2130261.0,7581.0,2016546.0
50011,30.0,13975.0,14005.0,131439.0,108883.0,22556.0,0.0,13975.0,0.0,0,0,0.0,0.0,0.0


In [12]:
# collective['تاريخ میلادی'] = pd.to_datetime(collective['تاريخ میلادی'])
group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective['تعداد کل روزهای بدهکار'] = group_by_cust['تاريخ میلادی'].diff()
group_by_cust = collective.groupby('کد ردیابی')
collective['تعداد کل روزهای بدهکار'] = group_by_cust['تعداد کل روزهای بدهکار'].shift(-1)

def assign_to_last(group, column):
    value = report_end_date - group.iloc[-1,group.columns.get_loc('تاريخ میلادی')]
    group.iloc[-1, group.columns.get_loc(column)] = value
    return group

group_by_cust = collective.groupby('کد ردیابی',group_keys=False)
collective = group_by_cust.apply(assign_to_last,column = 'تعداد کل روزهای بدهکار')

collective['مانده در خط تسویه شده'] = collective['مانده در خط'] - collective['مانده درخط تسویه نشده']
